In [1]:
import os
import pandas as pd
from datetime import date, datetime, timedelta

#### Loading 30 year data in csv/xls format exported from yearly data

In [2]:
 # Before running...data for 22h and 23h was added to the last excel file to make sure it has the same length as other files that are in the middle. 
cwd = os.path.abspath(r'D:\Weather_files\CORDEX_weather_data\MPI_REMO') 
files = os.listdir(cwd)

In [3]:
df = pd.DataFrame()
for file in files:
    if file.endswith('.xlsx'):
        df = df.append(pd.read_excel('D:/Weather_files/CORDEX_weather_data/MPI_REMO/'+file), ignore_index=True) 
df.head(1)

,DateTime,year,month,DB_Temp,DewPointTemperature,RelativeHumidity,WindSpeed,CloudCover,AtmosPressure,GHI,DNI,SolarZenith,DHI
0,2040-01-01,2040,1,5.669122,3.03,83.110458,4.562614,87.410179,100824.1875,0.0,0.0,155.76,0.0


In [4]:
# Because of three hour intervals and previous interpolations that covered the inner rows 
# two hours of data at 22h and 23h at the end of each year is missing. ffill is used to fill 2 nan cells for a year
df=df.set_index('DateTime').resample('H').last()
df = df.fillna(method='ffill')
# Removing leap day data if year is a leap year 
df = df[~((df.index.month == 2) & (df.index.day == 29))]
# Changing data type 
df = df.astype({"year":'int', "month":'int'})

In [53]:
# Seeing if joining datafiles have been done properly
df.iloc[8757:8762]

,year,month,DB_Temp,DewPointTemperature,RelativeHumidity,WindSpeed,CloudCover,AtmosPressure,GHI,DNI,SolarZenith,DHI
DateTime,,,,,,,,,,,,
2040-12-31 21:00:00,2040,12,8.466852,8.14,97.828827,2.687051,18.505352,101886.265625,0.0,0.0,135.23,0.0
2040-12-31 22:00:00,2040,12,8.466852,8.14,97.828827,2.687051,18.505352,101886.265625,0.0,0.0,135.23,0.0
2040-12-31 23:00:00,2040,12,8.466852,8.14,97.828827,2.687051,18.505352,101886.265625,0.0,0.0,135.23,0.0
2041-01-01 00:00:00,2041,1,10.870782,9.81,93.144684,4.460026,72.389999,101691.000000,0.0,0.0,155.70,0.0
2041-01-01 01:00:00,2041,1,14.041664,12.60,91.031720,4.644883,75.561045,101631.875000,0.0,0.0,153.75,0.0


In [6]:
# Average value of each variable 
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 271558 entries, 2040-01-01 00:00:00 to 2070-12-31 21:00:00
Data columns (total 12 columns):
year                   271558 non-null int32
month                  271558 non-null int32
DB_Temp                271558 non-null float64
DewPointTemperature    271558 non-null float64
RelativeHumidity       271558 non-null float64
WindSpeed              271558 non-null float64
CloudCover             271558 non-null float64
AtmosPressure          271558 non-null float64
GHI                    271558 non-null float64
DNI                    271558 non-null float64
SolarZenith            271558 non-null float64
DHI                    271558 non-null float64
dtypes: float64(10), int32(2)
memory usage: 24.9 MB


#### Calculating daily means

In [7]:
# For each climatic parameter p : (dry bulb temperature, humidity ratio, GHI) perform the following operatios
# Resampling to calcualte daily means (p^-) of the given parameters (DBT,DPT,WSP,GHI)
df_daily = df.resample('D').mean()

In [8]:
new_columns = ['DB_Temp','DewPointTemperature','RelativeHumidity', 'WindSpeed','GHI','DNI']
df_daily = df_daily[new_columns]
df_daily.head(2)

,DB_Temp,DewPointTemperature,RelativeHumidity,WindSpeed,GHI,DNI
DateTime,,,,,,
2040-01-01,9.025214,6.305,83.100938,6.126465,7.808098,0.372917
2040-01-02,8.077459,4.975,80.829152,5.217163,15.170530,2.211250


In [9]:
df_daily.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11323 entries, 2040-01-01 to 2070-12-31
Freq: D
Data columns (total 6 columns):
DB_Temp                11315 non-null float64
DewPointTemperature    11315 non-null float64
RelativeHumidity       11315 non-null float64
WindSpeed              11315 non-null float64
GHI                    11315 non-null float64
DNI                    11315 non-null float64
dtypes: float64(6)
memory usage: 619.2 KB


In [10]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
# Months dictionary 
monthDict = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',
             7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}

In [12]:
# A Dataframe to write out results 
AllYears = np.arange(df_daily.index[0].year,df_daily.index[len(df_daily.index)-1].year+1,1).tolist()
All_df = pd.DataFrame(index=AllYears)

#### Calculating FS statistics for DB Temp, RH, and GHI, plus mean wind speed deviation for each year to multiyear.

In [13]:
for Month in [1,2,3,4,5,6,7,8,9,10,11,12]:
    tempe = []
    RelHu = []
    HGIrr = []
    WindS = []
    for Year in AllYears:
        #### Temperature
        y = df_daily[(df_daily.index.month == Month)].DB_Temp.values
        y = y[~np.isnan(y)]
        h, b = np.histogram(y, bins=100, density=True)
        cdf = np.cumsum(h*np.diff(b))
        #
        y1 = df_daily[((df_daily.index.month == Month) & (df_daily.index.year == Year))].DB_Temp.values
        y1 = y1[~np.isnan(y1)]
        h1, b1 = np.histogram(y1, bins=100, density=True)
        cdf1 = np.cumsum(h1*np.diff(b1))
        ## Finkelstien-Schafar Statistics
        ts = np.sum(abs(cdf - cdf1))
        tempe.append(ts)
    
        #### Relative humidity 
        x = df_daily[(df_daily.index.month == Month)].RelativeHumidity.values
        x = x[~np.isnan(x)]
        j, a = np.histogram(x, bins=100, density=True)
        cdfRH = np.cumsum(j*np.diff(a))
        #
        x1 = df_daily[((df_daily.index.month == Month) & (df_daily.index.year == Year))].RelativeHumidity.values
        x1 = x1[~np.isnan(x1)]
        j1, a1 = np.histogram(x1, bins=100, density=True)
        cdfRH1 = np.cumsum(j1*np.diff(a1))
        ## Finkelstien-Schafar Statistics
        rs = np.sum(abs(cdfRH - cdfRH1))
        RelHu.append(rs)
    
        ####Global horizontal irradiance
        z = df_daily[(df_daily.index.month == Month)].GHI.values
        z = z[~np.isnan(z)]
        k, s = np.histogram(z, bins=100, density=True)
        cdfGHI = np.cumsum(k*np.diff(s))
        #
        z1 = df_daily[((df_daily.index.month == Month) & (df_daily.index.year == Year))].GHI.values
        z1 = z1[~np.isnan(z1)]
        k1, s1 = np.histogram(z1, bins=100, density=True)
        cdfGHI1 = np.cumsum(k1*np.diff(s1))
        ## Finkelstien-Schafar Statistics
        hs = np.sum(abs(cdfGHI - cdfGHI1))
        HGIrr.append(hs)
    
        #### Deviation of month's mean wind speed for a year from multiyear mean
        w = df_daily[(df_daily.index.month == Month)].WindSpeed.values
        w = w[~np.isnan(w)]
        WS = sum(w)/len(w)
        #
        w1 = df_daily[((df_daily.index.month == Month) & (df_daily.index.year == Year))].WindSpeed.values
        w1 = w1[~np.isnan(w1)]
        WS1 = sum(w1)/len(w1)
        ## Deviation of means 
        ws = abs(WS - WS1)
        WindS.append(ws)
        ###
    
    All_df['DBT_'+str(monthDict[Month])]=tempe
    All_df['RH_'+str(monthDict[Month])]=RelHu
    All_df['GHI_'+str(monthDict[Month])]=HGIrr
    All_df['WS_'+str(monthDict[Month])]=WindS

In [14]:
All_df.head(2)

,DBT_Jan,RH_Jan,GHI_Jan,WS_Jan,DBT_Feb,RH_Feb,GHI_Feb,WS_Feb,DBT_Mar,RH_Mar,...,GHI_Oct,WS_Oct,DBT_Nov,RH_Nov,GHI_Nov,WS_Nov,DBT_Dec,RH_Dec,GHI_Dec,WS_Dec
2040,11.308012,21.705515,9.312175,0.225798,10.269585,23.408986,7.342166,0.320906,6.113424,14.323621,...,10.753382,0.108384,11.365591,11.608602,5.680645,0.056894,13.368366,14.869927,7.276795,0.364859
2041,11.310094,25.964620,4.329865,0.223447,16.417051,13.667051,9.213134,0.104302,10.490114,15.064516,...,10.876171,0.210872,25.659140,10.037634,5.696774,0.572617,5.711759,15.937565,4.519251,0.743313


#### Ranking and selecting the best year for each month

In [15]:
##Ranking dry bulb temperature
All_df['R_DBT_Jan']=All_df['DBT_Jan'].rank(ascending=True)
All_df['R_DBT_Feb']=All_df['DBT_Feb'].rank(ascending=True)
All_df['R_DBT_Mar']=All_df['DBT_Mar'].rank(ascending=True)
All_df['R_DBT_Apr']=All_df['DBT_Apr'].rank(ascending=True)
All_df['R_DBT_May']=All_df['DBT_May'].rank(ascending=True)
All_df['R_DBT_Jun']=All_df['DBT_Jun'].rank(ascending=True)
All_df['R_DBT_Jul']=All_df['DBT_Jul'].rank(ascending=True)
All_df['R_DBT_Aug']=All_df['DBT_Aug'].rank(ascending=True)
All_df['R_DBT_Sep']=All_df['DBT_Sep'].rank(ascending=True)
All_df['R_DBT_Oct']=All_df['DBT_Oct'].rank(ascending=True)
All_df['R_DBT_Nov']=All_df['DBT_Nov'].rank(ascending=True)
All_df['R_DBT_Dec']=All_df['DBT_Dec'].rank(ascending=True)
##ranking relative humidity 
All_df['R_RH_Jan']=All_df['RH_Jan'].rank(ascending=True)
All_df['R_RH_Feb']=All_df['RH_Feb'].rank(ascending=True)
All_df['R_RH_Mar']=All_df['RH_Mar'].rank(ascending=True)
All_df['R_RH_Apr']=All_df['RH_Apr'].rank(ascending=True)
All_df['R_RH_May']=All_df['RH_May'].rank(ascending=True)
All_df['R_RH_Jun']=All_df['RH_Jun'].rank(ascending=True)
All_df['R_RH_Jul']=All_df['RH_Jul'].rank(ascending=True)
All_df['R_RH_Aug']=All_df['RH_Aug'].rank(ascending=True)
All_df['R_RH_Sep']=All_df['RH_Sep'].rank(ascending=True)
All_df['R_RH_Oct']=All_df['RH_Oct'].rank(ascending=True)
All_df['R_RH_Nov']=All_df['RH_Nov'].rank(ascending=True)
All_df['R_RH_Dec']=All_df['RH_Dec'].rank(ascending=True)
##ranking global horizontal irradiance
All_df['R_GHI_Jan']=All_df['GHI_Jan'].rank(ascending=True)
All_df['R_GHI_Feb']=All_df['GHI_Feb'].rank(ascending=True)
All_df['R_GHI_Mar']=All_df['GHI_Mar'].rank(ascending=True)
All_df['R_GHI_Apr']=All_df['GHI_Apr'].rank(ascending=True)
All_df['R_GHI_May']=All_df['GHI_May'].rank(ascending=True)
All_df['R_GHI_Jun']=All_df['GHI_Jun'].rank(ascending=True)
All_df['R_GHI_Jul']=All_df['GHI_Jul'].rank(ascending=True)
All_df['R_GHI_Aug']=All_df['GHI_Aug'].rank(ascending=True)
All_df['R_GHI_Sep']=All_df['GHI_Sep'].rank(ascending=True)
All_df['R_GHI_Oct']=All_df['GHI_Oct'].rank(ascending=True)
All_df['R_GHI_Nov']=All_df['GHI_Nov'].rank(ascending=True)
All_df['R_GHI_Dec']=All_df['GHI_Dec'].rank(ascending=True)

In [16]:
### Sum the FS value of three primary variables (DBT,RH,GHI) for each month

All_df['January']=All_df['R_DBT_Jan']+ All_df['R_RH_Jan']+ All_df['R_GHI_Jan']
All_df['February']=All_df['R_DBT_Feb']+ All_df['R_RH_Feb']+ All_df['R_GHI_Feb']
All_df['March']=All_df['R_DBT_Mar']+ All_df['R_RH_Mar']+ All_df['R_GHI_Mar']
All_df['April']=All_df['R_DBT_Apr']+ All_df['R_RH_Apr']+ All_df['R_GHI_Apr']
All_df['MAY']=All_df['R_DBT_May']+ All_df['R_RH_May']+ All_df['R_GHI_May']
All_df['June']=All_df['R_DBT_Jun']+ All_df['R_RH_Jun']+ All_df['R_GHI_Jun']
All_df['July']=All_df['R_DBT_Jul']+ All_df['R_RH_Jul']+ All_df['R_GHI_Jul']
All_df['August']=All_df['R_DBT_Aug']+ All_df['R_RH_Aug']+ All_df['R_GHI_Aug']
All_df['September']=All_df['R_DBT_Sep']+ All_df['R_RH_Sep']+ All_df['R_GHI_Sep']
All_df['October']=All_df['R_DBT_Oct']+ All_df['R_RH_Oct']+ All_df['R_GHI_Oct']
All_df['November']=All_df['R_DBT_Nov']+ All_df['R_RH_Nov']+ All_df['R_GHI_Nov']
All_df['December']=All_df['R_DBT_Dec']+ All_df['R_RH_Dec']+ All_df['R_GHI_Dec']

## Rank order the sum total of variables to identify three lowest sizes 
All_df['Jan']=All_df['January'].rank(method='first',ascending=True)
All_df['Feb']=All_df['February'].rank(method='first',ascending=True)
All_df['Mar']=All_df['March'].rank(method='first',ascending=True)
All_df['Apr']=All_df['April'].rank(method='first',ascending=True)
All_df['May']=All_df['MAY'].rank(method='first',ascending=True)
All_df['Jun']=All_df['June'].rank(method='first',ascending=True)
All_df['Jul']=All_df['July'].rank(method='first',ascending=True)
All_df['Aug']=All_df['August'].rank(method='first',ascending=True)
All_df['Sep']=All_df['September'].rank(method='first',ascending=True)
All_df['Oct']=All_df['October'].rank(method='first',ascending=True)
All_df['Nov']=All_df['November'].rank(method='first',ascending=True)
All_df['Dec']=All_df['December'].rank(method='first',ascending=True)

In [17]:
#colmn = All_df.columns.tolist()
#print (colmn)

In [18]:
all_columns = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep',
               'Oct','Nov','Dec','WS_Jan','WS_Feb','WS_Mar','WS_Apr',
               'WS_May','WS_Jun','WS_Jul','WS_Aug','WS_Sep',
               'WS_Oct','WS_Nov','WS_Dec']
df_MS = All_df[all_columns]

In [19]:
### Selecting year of TMY for eaxh month 
January = df_MS.index[df_MS['WS_Jan']==min(df_MS['WS_Jan']
                                           .loc[df_MS.index.isin(df_MS.index[df_MS['Jan']<4]
                                                                 .tolist())])].values[0]
February = df_MS.index[df_MS['WS_Feb']==min(df_MS['WS_Feb']
                                            .loc[df_MS.index.isin(df_MS.index[df_MS['Feb']<4]
                                                                  .tolist())])].values[0]
March = df_MS.index[df_MS['WS_Mar']==min(df_MS['WS_Mar']
                                         .loc[df_MS.index.isin(df_MS.index[df_MS['Mar']<4]
                                                               .tolist())])].values[0]
April = df_MS.index[df_MS['WS_Apr']==min(df_MS['WS_Apr']
                                         .loc[df_MS.index.isin(df_MS.index[df_MS['Apr']<4]
                                                               .tolist())])].values[0]
May = df_MS.index[df_MS['WS_May']==min(df_MS['WS_May']
                                       .loc[df_MS.index.isin(df_MS.index[df_MS['May']<4]
                                                             .tolist())])].values[0]
June = df_MS.index[df_MS['WS_Jun']==min(df_MS['WS_Jun']
                                        .loc[df_MS.index.isin(df_MS.index[df_MS['Jun']<4]
                                                              .tolist())])].values[0]
July = df_MS.index[df_MS['WS_Jul']==min(df_MS['WS_Jul']
                                        .loc[df_MS.index.isin(df_MS.index[df_MS['Jul']<4]
                                                              .tolist())])].values[0]
August = df_MS.index[df_MS['WS_Aug']==min(df_MS['WS_Aug']
                                          .loc[df_MS.index.isin(df_MS.index[df_MS['Aug']<4]
                                                                .tolist())])].values[0]
September = df_MS.index[df_MS['WS_Sep']==min(df_MS['WS_Sep']
                                             .loc[df_MS.index.isin(df_MS.index[df_MS['Sep']<4]
                                                                   .tolist())])].values[0]
October = df_MS.index[df_MS['WS_Oct']==min(df_MS['WS_Oct']
                                           .loc[df_MS.index.isin(df_MS.index[df_MS['Oct']<4]
                                                                 .tolist())])].values[0]
November = df_MS.index[df_MS['WS_Nov']==min(df_MS['WS_Nov']
                                            .loc[df_MS.index.isin(df_MS.index[df_MS['Nov']<4]
                                                                  .tolist())])].values[0]
December = df_MS.index[df_MS['WS_Dec']==min(df_MS['WS_Dec']
                                            .loc[df_MS.index.isin(df_MS.index[df_MS['Dec']<4]
                                                                  .tolist())])].values[0]

In [20]:
print('January = '+str(January))
print('February = '+str(February))
print('March = '+str(March))
print('April = '+str(April))
print('May = '+str(May))
print('June = '+str(June))
print('July = '+str(July))
print('August = '+str(August))
print('September = '+str(September))
print('October = '+str(October))
print('November = '+str(November))
print('December = '+str(December))

January = 2064
February = 2068
March = 2047
April = 2051
May = 2067
June = 2042
July = 2045
August = 2060
September = 2047
October = 2048
November = 2040
December = 2065


In [21]:
import warnings
warnings.filterwarnings("ignore")

#### Removing first and last 8 hours of data of each month and interpolating them linearly to smooth out transition

In [22]:
#### 
df1 = df[((df.index.month == 1) & (df.index.year == January))]
# last 7 hours of the month
df1['DB_Temp'].iloc[-7:]=np.nan
df1['CloudCover'].iloc[-7:]=np.nan
df1['WindSpeed'].iloc[-7:]=np.nan
df1['RelativeHumidity'].iloc[-7:]=np.nan
df1['DewPointTemperature'].iloc[-7:]=np.nan
######
df2 = df[((df.index.month == 2) & (df.index.year == February))]
# first 8 hours of the month
df2['DB_Temp'].iloc[:7]=np.nan
df2['CloudCover'].iloc[:7]=np.nan
df2['WindSpeed'].iloc[:7]=np.nan
df2['RelativeHumidity'].iloc[:7]=np.nan
df2['DewPointTemperature'].iloc[:7]=np.nan
# last 8 hours of the month
df2['DB_Temp'].iloc[-7:]=np.nan
df2['CloudCover'].iloc[-7:]=np.nan
df2['WindSpeed'].iloc[-7:]=np.nan
df2['RelativeHumidity'].iloc[-7:]=np.nan
df2['DewPointTemperature'].iloc[-7:]=np.nan
######
df3 = df[((df.index.month == 3) & (df.index.year == March))]
# first 8 hours of the month
df3['DB_Temp'].iloc[:7]=np.nan
df3['CloudCover'].iloc[:7]=np.nan
df3['WindSpeed'].iloc[:7]=np.nan
df3['RelativeHumidity'].iloc[:7]=np.nan
df3['DewPointTemperature'].iloc[:7]=np.nan
# last 8 hours of the month 
df3['DB_Temp'].iloc[-7:]=np.nan
df3['CloudCover'].iloc[-7:]=np.nan
df3['WindSpeed'].iloc[-7:]=np.nan
df3['RelativeHumidity'].iloc[-7:]=np.nan
df3['DewPointTemperature'].iloc[-7:]=np.nan
######
df4 = df[((df.index.month == 4) & (df.index.year == April))]
# first 8 hours of the month
df4['DB_Temp'].iloc[:7]=np.nan
df4['CloudCover'].iloc[:7]=np.nan
df4['WindSpeed'].iloc[:7]=np.nan
df4['RelativeHumidity'].iloc[:7]=np.nan
df4['DewPointTemperature'].iloc[:7]=np.nan
# last 8 hours of the month 
df4['DB_Temp'].iloc[-7:]=np.nan
df4['CloudCover'].iloc[-7:]=np.nan
df4['WindSpeed'].iloc[-7:]=np.nan
df4['RelativeHumidity'].iloc[-7:]=np.nan
df4['DewPointTemperature'].iloc[-7:]=np.nan
######
df5 = df[((df.index.month == 5) & (df.index.year == May))]
# first 8 hours of the month
df5['DB_Temp'].iloc[:7]=np.nan
df5['CloudCover'].iloc[:7]=np.nan
df5['WindSpeed'].iloc[:7]=np.nan
df5['RelativeHumidity'].iloc[:7]=np.nan
df5['DewPointTemperature'].iloc[:7]=np.nan
# last 8 hours of the month 
df5['DB_Temp'].iloc[-7:]=np.nan
df5['CloudCover'].iloc[-7:]=np.nan
df5['WindSpeed'].iloc[-7:]=np.nan
df5['RelativeHumidity'].iloc[-7:]=np.nan
df5['DewPointTemperature'].iloc[-7:]=np.nan
######
df6 = df[((df.index.month == 6) & (df.index.year == June))]
# first 8 hours of the month
df6['DB_Temp'].iloc[:7]=np.nan
df6['CloudCover'].iloc[:7]=np.nan
df6['WindSpeed'].iloc[:7]=np.nan
df6['RelativeHumidity'].iloc[:7]=np.nan
df6['DewPointTemperature'].iloc[:7]=np.nan
# last 8 hours of the month 
df6['DB_Temp'].iloc[-7:]=np.nan
df6['CloudCover'].iloc[-7:]=np.nan
df6['WindSpeed'].iloc[-7:]=np.nan
df6['RelativeHumidity'].iloc[-7:]=np.nan
df6['DewPointTemperature'].iloc[-7:]=np.nan
######
df7 = df[((df.index.month == 7) & (df.index.year == July))]
# first 8 hours of the month
df7['DB_Temp'].iloc[:7]=np.nan
df7['CloudCover'].iloc[:7]=np.nan
df7['WindSpeed'].iloc[:7]=np.nan
df7['RelativeHumidity'].iloc[:7]=np.nan
df7['DewPointTemperature'].iloc[:7]=np.nan
# last 8 hours of the month 
df7['DB_Temp'].iloc[-7:]=np.nan
df7['CloudCover'].iloc[-7:]=np.nan
df7['WindSpeed'].iloc[-7:]=np.nan
df7['RelativeHumidity'].iloc[-7:]=np.nan
df7['DewPointTemperature'].iloc[-7:]=np.nan
######
df8 = df[((df.index.month == 8) & (df.index.year == August))]
# first 8 hours of the month
df8['DB_Temp'].iloc[:7]=np.nan
df8['CloudCover'].iloc[:7]=np.nan
df8['WindSpeed'].iloc[:7]=np.nan
df8['RelativeHumidity'].iloc[:7]=np.nan
df8['DewPointTemperature'].iloc[:7]=np.nan
# last 8 hours of the month 
df8['DB_Temp'].iloc[-7:]=np.nan
df8['CloudCover'].iloc[-7:]=np.nan
df8['WindSpeed'].iloc[-7:]=np.nan
df8['RelativeHumidity'].iloc[-7:]=np.nan
df8['DewPointTemperature'].iloc[-7:]=np.nan
######
df9 = df[((df.index.month == 9) & (df.index.year == September))]
# first 8 hours of the month
df9['DB_Temp'].iloc[:7]=np.nan
df9['CloudCover'].iloc[:7]=np.nan
df9['WindSpeed'].iloc[:7]=np.nan
df9['RelativeHumidity'].iloc[:7]=np.nan
df9['DewPointTemperature'].iloc[:7]=np.nan
# last 8 hours of the month 
df9['DB_Temp'].iloc[-7:]=np.nan
df9['CloudCover'].iloc[-7:]=np.nan
df9['WindSpeed'].iloc[-7:]=np.nan
df9['RelativeHumidity'].iloc[-7:]=np.nan
df9['DewPointTemperature'].iloc[-7:]=np.nan
######
df10 = df[((df.index.month == 10) & (df.index.year == October))]
# first 8 hours of the month
df10['DB_Temp'].iloc[:7]=np.nan
df10['CloudCover'].iloc[:7]=np.nan
df10['WindSpeed'].iloc[:7]=np.nan
df10['RelativeHumidity'].iloc[:7]=np.nan
df10['DewPointTemperature'].iloc[:7]=np.nan
# last 8 hours of the month 
df10['DB_Temp'].iloc[-7:]=np.nan
df10['CloudCover'].iloc[-7:]=np.nan
df10['WindSpeed'].iloc[-7:]=np.nan
df10['RelativeHumidity'].iloc[-7:]=np.nan
df10['DewPointTemperature'].iloc[-7:]=np.nan
######
df11 = df[((df.index.month == 11) & (df.index.year == November))]
# first 8 hours of the month
df11['DB_Temp'].iloc[:7]=np.nan
df11['CloudCover'].iloc[:7]=np.nan
df11['WindSpeed'].iloc[:7]=np.nan
df11['RelativeHumidity'].iloc[:7]=np.nan
df11['DewPointTemperature'].iloc[:7]=np.nan
# last 8 hours of the month 
df11['DB_Temp'].iloc[-7:]=np.nan
df11['CloudCover'].iloc[-7:]=np.nan
df11['WindSpeed'].iloc[-7:]=np.nan
df11['RelativeHumidity'].iloc[-7:]=np.nan
df11['DewPointTemperature'].iloc[-7:]=np.nan
######
df12 = df[((df.index.month == 12) & (df.index.year == December))]
# first 8 hours of the month
df12['DB_Temp'].iloc[:7]=np.nan
df12['CloudCover'].iloc[:7]=np.nan
df12['WindSpeed'].iloc[:7]=np.nan
df12['RelativeHumidity'].iloc[:7]=np.nan
df12['DewPointTemperature'].iloc[:7]=np.nan

Frames = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12]
result = pd.concat(Frames)
result = result.interpolate()

In [54]:
result.iloc[4446:4448]

,year,month,DB_Temp,DewPointTemperature,RelativeHumidity,WindSpeed,CloudCover,AtmosPressure,GHI,DNI,SolarZenith,DHI
DateTime,,,,,,,,,,,,
2045-07-05 06:00:00,2045,7,16.187585,13.62,84.758759,2.632964,74.367569,102383.687500,63.283680,3.14,75.29,60.16
2045-07-05 07:00:00,2045,7,16.964542,13.35,79.273621,2.928851,68.954421,102389.856771,140.531202,18.89,65.37,156.08


In [24]:
result['DHI'] = result['DHI'].apply(lambda x : x if x > 0 else 0)

In [26]:
result.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8760 entries, 2064-01-01 00:00:00 to 2065-12-31 23:00:00
Data columns (total 12 columns):
year                   8760 non-null int32
month                  8760 non-null int32
DB_Temp                8760 non-null float64
DewPointTemperature    8760 non-null float64
RelativeHumidity       8760 non-null float64
WindSpeed              8760 non-null float64
CloudCover             8760 non-null float64
AtmosPressure          8760 non-null float64
GHI                    8760 non-null float64
DNI                    8760 non-null float64
SolarZenith            8760 non-null float64
DHI                    8760 non-null float64
dtypes: float64(10), int32(2)
memory usage: 821.2 KB


In [56]:
result.head(3)

,year,month,DB_Temp,DewPointTemperature,RelativeHumidity,WindSpeed,CloudCover,AtmosPressure,GHI,DNI,SolarZenith,DHI
DateTime,,,,,,,,,,,,
2064-01-01 00:00:00,2064,1,12.984369,12.08,94.240639,7.898614,100.0,99949.273438,0.0,0.0,155.75,0.0
2064-01-01 01:00:00,2064,1,12.483222,11.62,94.461810,7.677654,100.0,99883.575521,0.0,0.0,153.77,0.0
2064-01-01 02:00:00,2064,1,12.673921,11.84,94.682981,7.456694,100.0,99817.877604,0.0,0.0,147.22,0.0


#### Overwriting calculated data to an existing epw file and saving it

In [29]:
## Writing the weather data on epw weather template downloaded from EnergyPlus weather data platform
from ladybug.epw import EPW
from pandas import DataFrame
from collections import OrderedDict
%config Completer.use_jedi = False

In [30]:
#field_number: a value between 0 to 34 for different available epw fields.
#            0 Year
#            1 Month
#            2 Day
#            3 Hour
#            4 Minute

#            6 Dry Bulb Temperature
#            7 Dew Point Temperature
#            8 Relative Humidity
#            9 Atmospheric Station Pressure
#            10 Extraterrestrial Horizontal Radiation
#            11 Extraterrestrial Direct Normal Radiation
#            12 Horizontal Infrared Radiation Intensity
#            13 Global Horizontal Radiation
#            14 Direct Normal Radiation
#            15 Diffuse Horizontal Radiation
#            16 Global Horizontal Illuminance
#            17 Direct Normal Illuminance
#            18 Diffuse Horizontal Illuminance
#            19 Zenith Luminance
#            20 Wind Direction
#            21 Wind Speed
#            22 Total Sky Cover
#            23 Opaque Sky Cover
#            24 Visibility
#            25 Ceiling Height
#            26 Present Weather Observation
#            27 Present Weather Codes
#            28 Precipitable Water
#            29 Aerosol Optical Depth
#            30 Snow Depth
#            31 Days Since Last Snowfall
#            32 Albedo
#            33 Liquid Precipitation Depth
#            34 Liquid Precipitation Quantity

In [31]:
def changeEPWData(oldEpwFilePath,newEpwFilePath,dataIndex,dataList):
    with open(oldEpwFilePath) as oldStream,open(newEpwFilePath,"w") as newStream:
        numCount=0
        for idx,lines in enumerate(oldStream):
            if lines.strip():
                try:
                    lineSplit=lines.strip().split(",")
                    dataTest=float(lineSplit[0])
                    lineSplit[dataIndex]=str(dataList[numCount])
                    data=",".join(lineSplit)
                    newStream.write(data+"\n")
                    numCount+=1
                except ValueError:
                    newStream.write(lines.strip()+"\n")
            else:
                newStream.write(lines)
    return newEpwFilePath

In [32]:
# Writing Dry bulb temperature on epw file. 
changeEPWData(r'D:\Weather_files\Nantes-RCP8-5_2050.epw',
              r'D:\Weather_files\CORDEX_weather_data\MPI_REMO\Temporary\CNRM2040_2070T.epw',
              dataIndex=6,
              dataList=result.DB_Temp)

'D:\\Weather_files\\CORDEX_weather_data\\MPI_REMO\\Temporary\\CNRM2040_2070T.epw'

In [33]:
# Overwriting year
changeEPWData(r'D:\Weather_files\CORDEX_weather_data\MPI_REMO\Temporary\CNRM2040_2070T.epw',
              r'D:\Weather_files\CORDEX_weather_data\MPI_REMO\Temporary\CNRM2040_2070Y.epw',
              dataIndex=0,
              dataList=result.year)

'D:\\Weather_files\\CORDEX_weather_data\\MPI_REMO\\Temporary\\CNRM2040_2070Y.epw'

In [34]:
# Overwriting dew point temperature
changeEPWData(r'D:\Weather_files\CORDEX_weather_data\MPI_REMO\Temporary\CNRM2040_2070Y.epw',
              r'D:\Weather_files\CORDEX_weather_data\MPI_REMO\Temporary\CNRM2040_2070DP.epw',
              dataIndex=7,
              dataList=result.DewPointTemperature)

'D:\\Weather_files\\CORDEX_weather_data\\MPI_REMO\\Temporary\\CNRM2040_2070DP.epw'

In [35]:
# Overwriting RH
changeEPWData(r'D:\Weather_files\CORDEX_weather_data\MPI_REMO\Temporary\CNRM2040_2070DP.epw',
              r'D:\Weather_files\CORDEX_weather_data\MPI_REMO\Temporary\CNRM2040_2070RH.epw',
              dataIndex=8,
              dataList=result.RelativeHumidity)

'D:\\Weather_files\\CORDEX_weather_data\\MPI_REMO\\Temporary\\CNRM2040_2070RH.epw'

In [36]:
# Overwriting Atmospheric pressure 
changeEPWData(r'D:\Weather_files\CORDEX_weather_data\MPI_REMO\Temporary\CNRM2040_2070RH.epw',
              r'D:\Weather_files\CORDEX_weather_data\MPI_REMO\Temporary\CNRM2040_2070AP.epw',
              dataIndex=9,
              dataList=result.AtmosPressure)

'D:\\Weather_files\\CORDEX_weather_data\\MPI_REMO\\Temporary\\CNRM2040_2070AP.epw'

In [37]:
# Overwriting GHI
changeEPWData(r'D:\Weather_files\CORDEX_weather_data\MPI_REMO\Temporary\CNRM2040_2070AP.epw',
              r'D:\Weather_files\CORDEX_weather_data\MPI_REMO\Temporary\CNRM2040_2070GHI.epw',
              dataIndex=13,
              dataList=result.GHI)

'D:\\Weather_files\\CORDEX_weather_data\\MPI_REMO\\Temporary\\CNRM2040_2070GHI.epw'

In [38]:
# Overwriting DNI
changeEPWData(r'D:\Weather_files\CORDEX_weather_data\MPI_REMO\Temporary\CNRM2040_2070GHI.epw',
              r'D:\Weather_files\CORDEX_weather_data\MPI_REMO\Temporary\CNRM2040_2070DNI.epw',
              dataIndex=14,
              dataList=result.DNI)

'D:\\Weather_files\\CORDEX_weather_data\\MPI_REMO\\Temporary\\CNRM2040_2070DNI.epw'

In [39]:
# Overwriting DHI
changeEPWData(r'D:\Weather_files\CORDEX_weather_data\MPI_REMO\Temporary\CNRM2040_2070DNI.epw',
              r'D:\Weather_files\CORDEX_weather_data\MPI_REMO\Temporary\CNRM2040_2070DHI.epw',
              dataIndex=15,
              dataList=result.DHI)

'D:\\Weather_files\\CORDEX_weather_data\\MPI_REMO\\Temporary\\CNRM2040_2070DHI.epw'

In [40]:
# Overwriting wind speed 
changeEPWData(r'D:\Weather_files\CORDEX_weather_data\MPI_REMO\Temporary\CNRM2040_2070DHI.epw',
              r'D:\Weather_files\MPI_REMO_deleteMe.epw',
              dataIndex=21,
              dataList=result.WindSpeed)

'D:\\Weather_files\\MPI_REMO_deleteMe.epw'

In [47]:
## Export result as csv 
#result.to_csv(r'D:\Weather_files\CORDEX_weather_data\CSV_IPSL2040_2070.csv',index = 0)